### Instead of a raw difference of spread as we did in NB1(i.e spread = sym1-sym2) we will try to find hedged OLS namely spread = sym1 - (α + β*sym2) where α is intercept and β is hedge ration/ slop (how much sym1 typically moves per unit mover of sym2)

In [2]:
import os
from datetime import datetime, timedelta, timezone

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import MetaTrader5 as mt5

In [ ]:

# initialize 
authorized = mt5.initialize()
print("initialize:", authorized, mt5.last_error())


# 3) quick check
info = mt5.account_info()
#info

initialize: True (1, 'Success')


In [4]:
SYM1 = "USTEC"#"GS.NYSE"#"NVDA.NAS"
SYM2 = "US500"#"MS.NYSE"#"AMD.NAS"
TF = mt5.TIMEFRAME_H1 # try "H4" or "D1" later
START = datetime(2025, 1, 1) # UTC
END = datetime.now() # UTC

In [5]:
# get data
s1_rates = mt5.copy_rates_range(SYM1,TF,START, END)
s1_rates = pd.DataFrame(s1_rates)
s1_rates["time"] = pd.to_datetime(s1_rates["time"], unit = "s")
print(s1_rates)
s2_rates = mt5.copy_rates_range(SYM2,TF,START, END)
s2_rates = pd.DataFrame(s2_rates)
s2_rates["time"] = pd.to_datetime(s2_rates["time"], unit = "s")


                    time      open      high       low     close  tick_volume  \
0    2024-12-31 23:00:00  20959.67  20989.72  20938.22  20974.10         7488   
1    2025-01-02 01:00:00  21009.44  21067.19  20991.19  21013.57         5127   
2    2025-01-02 02:00:00  21013.57  21014.19  20869.19  20977.94         8891   
3    2025-01-02 03:00:00  20978.07  21058.82  20978.07  21041.82         9340   
4    2025-01-02 04:00:00  21040.82  21079.32  21034.82  21069.07         6326   
...                  ...       ...       ...       ...       ...          ...   
4668 2025-10-16 09:00:00  24781.30  24855.10  24781.10  24852.90        11473   
4669 2025-10-16 10:00:00  24852.40  24883.50  24801.10  24867.30        16407   
4670 2025-10-16 11:00:00  24867.30  24883.90  24833.40  24847.10        13928   
4671 2025-10-16 12:00:00  24847.10  24873.50  24833.90  24869.40        10068   
4672 2025-10-16 13:00:00  24869.50  24899.10  24867.10  24878.60         7630   

      spread  real_volume  

In [6]:
df = s1_rates[["time", "open"]].merge(s2_rates[["time", "open"]], on = "time")

df = df.rename(columns={"open_x": SYM1, "open_y": SYM2})
print(df)

                    time     USTEC   US500
0    2024-12-31 23:00:00  20959.67  5881.5
1    2025-01-02 01:00:00  21009.44  5893.6
2    2025-01-02 02:00:00  21013.57  5887.3
3    2025-01-02 03:00:00  20978.07  5882.9
4    2025-01-02 04:00:00  21040.82  5894.6
...                  ...       ...     ...
4667 2025-10-16 09:00:00  24781.30  6675.4
4668 2025-10-16 10:00:00  24852.40  6688.9
4669 2025-10-16 11:00:00  24867.30  6691.3
4670 2025-10-16 12:00:00  24847.10  6688.5
4671 2025-10-16 13:00:00  24869.50  6695.8

[4672 rows x 3 columns]


In [7]:
# plot the df
import plotly.express as px
fig = px.line(df, x ="time", y = [SYM1, SYM2], title = f"Historical Prices - {SYM1} vs {SYM2} (based on mt5 {TF} timeframe infos)")

fig.show()

In [8]:
# investigate correlation using pandas method
df [[SYM1,SYM2]].corr()
# 99% correlation for xbr-xti

,USTEC,US500
USTEC,1.000000,0.994574
US500,0.994574,1.000000


# Doing OLS and finding static alpha and beta over some period(below we will find alpha and beta over rolling window to always update and capture changes)

### now we’ll switch from the raw difference to the OLS-hedged residual and then compute Bollinger/z on that.

#### we estimate the sym1 = α + β*sym2 and the define the spread as the residual spread = sym1 - (α + β*sym2). Then compute the SMA/standard_dev/Bollinger bands on spread and not on sym1-sym2 like in NB1


In [9]:
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

# prepare the arrays (float, finite)
x = df[SYM2].astype(float).to_numpy()
x
y = df[SYM1].astype(float).to_numpy()

#mask = np.isfinite(x) & np.isfinite(y) # we dont' really need this since it just checks if x,y are finite and we know they are finite since we have start and end date

# OLS: SYM1 ≈ α + β*SYM2  (with intercept)
X = sm.add_constant(x)    # [1, x]
res = sm.OLS(y, X).fit(cov_type="HAC", cov_kwds={"maxlags":5})  # HAC SEs (Newey–West)

print(res.summary())
# resul mean that the slope β is 0.9200 namely sym1 moves ~1-for-0.9 with sym2 on average
# the intercept α is 25.011 namely sym1 tends to sit about 25$ above β*sym2 on average
#R^2 = 0.942
# Durbin–Watson: residuals are highly autocorrelated (expected when regressing levels of assets)
# HAC SEs: The t/z values in the table are HAC-robust (Newey–West). This doesn’t change α,β — only their standard errors and thus the test stats/p-values are robust to heteroskedasticity & autocorrelation, which you clearly have (DW≈0).

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.989
Model:                            OLS   Adj. R-squared:                  0.989
Method:                 Least Squares   F-statistic:                 8.777e+04
Date:                Sun, 19 Oct 2025   Prob (F-statistic):               0.00
Time:                        21:12:09   Log-Likelihood:                -31148.
No. Observations:                4672   AIC:                         6.230e+04
Df Residuals:                    4670   BIC:                         6.231e+04
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const      -6312.9088     97.372    -64.833      0.0

In [10]:
print(res.params) #[25.01192506  0.91997332]


[-6.31290875e+03  4.64120523e+00]


### plot the OLS line

In [11]:
import plotly.graph_objects as go
import plotly.express as px

alpha = float(res.params[0])
beta = float(res.params[1])

# fitted values and residuals (for convenience/diagnostics)
df["fitted"] = alpha + beta * df[SYM2]
df["residual"] = df[SYM1] - df["fitted"]

# Scatter of points + OLS line
x_min, x_max = df[SYM2].min(), df[SYM2].max()
line_x = np.array([x_min, x_max])
line_y = alpha + beta * line_x

fig = go.Figure()
fig.add_trace(go.Scattergl(
    x=df[SYM2], y=df[SYM1],
    mode="markers", name="data",
    marker=dict(size=6, opacity=0.3)
))
fig.add_trace(go.Scatter(
    x=line_x, y=line_y,
    mode="lines", name=f"OLS fit: y = {alpha:.2f} + {beta:.3f}·x"
))
title_text = (f"Regression line: {SYM1} ≈ α + β·{SYM2}<br>"
    f"α and β are computed based on the fix period of {START} and {END}")
fig.update_layout(
    title=title_text,
    xaxis_title=SYM2, yaxis_title=SYM1
)
fig.show()

### so that means the hedged spread(residual) is SYM1​−(alpha + beta⋅SYM2​) . Thats what we will pass into the rolling SMA/σ/Bollinger and z-score logic

### let’s compute the residual and verify stationarity

In [12]:
from statsmodels.tsa.stattools import adfuller
# Residual / hedged spread
df["spread"] = df[SYM1] - (res.params[0] + res.params[1] * df[SYM2])

# ADF test on residual (Engle–Granger step 2)

adf_stat, pval, lags, nobs, crit, _ = adfuller(df["spread"].values, regression="c", autolag="AIC")
print(f"ADF(residual): stat={adf_stat:.3f}, p={pval:.4f}, lags={lags}, nobs={nobs}")
print(f"Critical values: {crit}")

ADF(residual): stat=-2.903, p=0.0450, lags=1, nobs=4670
Critical values: {'1%': np.float64(-3.4317510488382816), '5%': np.float64(-2.8621591024569732), '10%': np.float64(-2.567099550642169)}


### Result: 
### When z > 0 (ε above its mean): SYM1 is rich vs the β-hedged SYM2 → pairs trade is typically short SYM1, long β·SYM2
### When z < 0 (ε below mean): SYM1 is cheap → long SYM1, short β·SYM2.
### Entry with |z| > Z_in(eg. 2.0) , exit with |z|< Z_out (eg.0.35)

# Now we will find alpha and beta on a rolling window and not on fixed period like we did above in order to gradually capture the changes happing.

In [13]:
import statsmodels.api as sm
# Note: Above we estimated a FIXED (α, β) on a specific period (e.g., 01/2025–09/2025).
# Now we compute a ROLLING (α, β) on a past-only window to adapt to gradual relationship changes.

# --- params ---
beta_window = 200   # lookback bars for rolling α,β (try 120–250 on D1)
band_window = 200   # rolling window for Bollinger/z on the residual
k = 2.0             # Bollinger multiplier (visual parity)

# pull series as float arrays
x = df[SYM2].astype(float).to_numpy()
y = df[SYM1].astype(float).to_numpy()
n = len(df)

# containers
alpha_roll = np.full(n, np.nan, dtype=float)
beta_roll  = np.full(n, np.nan, dtype=float)

total_models = n - beta_window # to count how many ols models we will tain in the for loop

# estimate α,β on [i-beta_window, i) and apply to point i (PAST-ONLY ⇒ no look-ahead)
for i in range(beta_window, n):
    xs = x[i - beta_window:i]
    ys = y[i - beta_window:i]
    Xw = sm.add_constant(xs)           # [1, x]
    ols_model = sm.OLS(ys, Xw).fit()        # classic OLS per window; SEs not used here
    alpha_roll[i] = float(ols_model.params[0])
    beta_roll[i]  = float(ols_model.params[1])
    if ((i - beta_window + 1) % 100 == 0) or (i == n - 1):
        print(f"Trained {i - beta_window + 1}/{total_models} OLS windows")


Trained 100/4472 OLS windows
Trained 200/4472 OLS windows
Trained 300/4472 OLS windows
Trained 400/4472 OLS windows
Trained 500/4472 OLS windows
Trained 600/4472 OLS windows
Trained 700/4472 OLS windows
Trained 800/4472 OLS windows
Trained 900/4472 OLS windows
Trained 1000/4472 OLS windows
Trained 1100/4472 OLS windows
Trained 1200/4472 OLS windows
Trained 1300/4472 OLS windows
Trained 1400/4472 OLS windows
Trained 1500/4472 OLS windows
Trained 1600/4472 OLS windows
Trained 1700/4472 OLS windows
Trained 1800/4472 OLS windows
Trained 1900/4472 OLS windows
Trained 2000/4472 OLS windows
Trained 2100/4472 OLS windows
Trained 2200/4472 OLS windows
Trained 2300/4472 OLS windows
Trained 2400/4472 OLS windows
Trained 2500/4472 OLS windows
Trained 2600/4472 OLS windows
Trained 2700/4472 OLS windows
Trained 2800/4472 OLS windows
Trained 2900/4472 OLS windows
Trained 3000/4472 OLS windows
Trained 3100/4472 OLS windows
Trained 3200/4472 OLS windows
Trained 3300/4472 OLS windows
Trained 3400/4472 O

In [14]:
# attach rolling α,β to df
df["alpha"] = alpha_roll
df["beta"]  = beta_roll


# rolling residual (hedged spread) using rolling α,β
df["spread"] = df[SYM1] - (df["alpha"] + df["beta"] * df[SYM2])
df['raw_difference'] = df[SYM1] - df[SYM2]

# Bollinger bands & z-score on the rolling residual
roll = df["spread"].rolling(band_window)
df["sma"] = roll.mean()
df["standard_deviation"] = roll.std()  # pandas sample std (ddof=1)
df["lower_band"] = df["sma"] - k * df["standard_deviation"]
df["upper_band"] = df["sma"] + k * df["standard_deviation"]
df["zscore"] = (df["spread"] - df["sma"]) / df["standard_deviation"]

df.tail(100)

,time,USTEC,US500,fitted,residual,spread,alpha,beta,raw_difference,sma,standard_deviation,lower_band,upper_band,zscore
4572,2025-10-10 06:00:00,25150.2,6744.7,24990.628167,159.571833,129.845535,-9582.178144,5.130329,18405.5,20.638401,47.242197,-73.845992,115.122794,2.311644
4573,2025-10-10 07:00:00,25129.1,6741.5,24975.776311,153.323689,123.991206,-9675.392471,5.144330,18387.6,21.413605,47.659975,-73.906346,116.733555,2.152280
4574,2025-10-10 08:00:00,25119.6,6738.5,24961.852695,157.747305,128.877308,-9748.995249,5.155408,18381.1,22.145517,48.180451,-74.215385,118.506418,2.215251
4575,2025-10-10 09:00:00,25145.6,6743.7,24985.986962,159.613038,127.001954,-9823.991796,5.166687,18401.9,22.871615,48.660967,-74.450318,120.193549,2.139915
4576,2025-10-10 10:00:00,25117.5,6738.0,24959.532092,157.967908,127.292216,-9915.936977,5.180490,18379.5,23.146800,49.095324,-75.043848,121.337448,2.121290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4667,2025-10-16 09:00:00,24781.3,6675.4,24668.992645,112.307355,21.620309,-1217.444222,3.891471,18105.9,41.940874,65.785652,-89.630430,173.512179,-0.308891
4668,2025-10-16 10:00:00,24852.4,6688.9,24731.648915,120.751085,39.933626,-1218.970694,3.891737,18163.5,42.308407,65.566620,-88.824833,173.441646,-0.036219
4669,2025-10-16 11:00:00,24867.3,6691.3,24742.787808,124.512192,44.914536,-1250.122013,3.896479,18176.0,42.951991,64.951369,-86.950747,172.854729,0.030216
4670,2025-10-16 12:00:00,24847.1,6688.5,24729.792433,117.307567,34.897900,-1327.365047,3.908136,18158.6,43.491119,64.430089,-85.369058,172.351296,-0.133373


In [15]:
px.line(df, x ="time", y = ["spread", "raw_difference"], title = f"Hedged Spread (i.e.Residual) vs Raw difference: {SYM1} vs {SYM2} (based on mt5 {TF} timeframe infos)")

In [16]:
#  clear warm-up region (no α,β or incomplete bands)
warmup = max(beta_window, band_window)
df.loc[:warmup, ["spread","sma","standard_deviation","lower_band","upper_band","zscore"]] = np.nan

print(f"Rolling α,β computed with statsmodels OLS. beta_window={beta_window}, band_window={band_window}, k={k}")
#df[["time", SYM1, SYM2, "alpha", "beta", "spread", "sma", "lower_band", "upper_band", "zscore"]].tail()

Rolling α,β computed with statsmodels OLS. beta_window=200, band_window=200, k=2.0


# PLOTS


In [17]:
# tiny signal logic just for plotting markers 

# %%
# Visual thresholds for entries/exits (adjust if you like)
Z_IN  = 2    # enter when |z| > Z_IN
Z_OUT = 0.5    # exit when |z| < Z_OUT

# Build simple entry/exit markers (state machine) – for visualization only
entries_time, entries_y, entries_text = [], [], []
exits_time,   exits_y,   exits_text   = [], [], []

pos = 0  # +1 long spread, -1 short spread, 0 flat
for i, r in df.iterrows():
    z = r.get("zscore", float("nan"))
    if not np.isfinite(z):
        continue

    if pos == 0:
        if z >= Z_IN:
            pos = -1
            entries_time.append(r["time"]); entries_y.append(r["spread"]); entries_text.append(f"Enter SHORT z={z:.2f}")
        elif z <= -Z_IN:
            pos = +1
            entries_time.append(r["time"]); entries_y.append(r["spread"]); entries_text.append(f"Enter LONG z={z:.2f}")
    else:
        if abs(z) < Z_OUT:
            exits_time.append(r["time"]); exits_y.append(r["spread"]); exits_text.append(f"Exit z={z:.2f}")
            pos = 0


## Spread + SMA + Bands + Markers (single plot)

In [18]:
# %%
import plotly.graph_objects as go

title_text = (
    f"Pairs Trading View – Hedged Residual Spread<br>"
    f"{SYM1} vs {SYM2} · TF={TF} · Bands on ε=SYM1−(α+β·SYM2)"
)

fig = go.Figure()

# spread & bands
fig.add_trace(go.Scatter(x=df["time"], y=df["spread"], name="spread (ε)", mode="lines"))
fig.add_trace(go.Scatter(x=df["time"], y=df["sma"], name="sma", mode="lines"))
fig.add_trace(go.Scatter(x=df["time"], y=df["upper_band"], name="upper_band", mode="lines", line=dict(dash="dash")))
fig.add_trace(go.Scatter(x=df["time"], y=df["lower_band"], name="lower_band", mode="lines", line=dict(dash="dash")))

# entry / exit markers (if any)
if entries_time:
    fig.add_trace(go.Scatter(x=entries_time, y=entries_y, mode="markers", name="entries",
                             marker=dict(symbol="circle", size=8), text=entries_text, hoverinfo="text+x+y"))
if exits_time:
    fig.add_trace(go.Scatter(x=exits_time, y=exits_y, mode="markers", name="exits",
                             marker=dict(symbol="triangle-up", size=9), text=exits_text, hoverinfo="text+x+y"))

fig.update_layout(
    title=dict(text=title_text, x=0.5),
    xaxis_title="time",
    yaxis_title="residual ε",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)
fig.show()


## z-score plot with thresholds

In [19]:
# %%
import plotly.express as px

fig_z = px.line(df, x="time", y="zscore",
                title=f"Residual z-score with thresholds · Z_IN={Z_IN}, Z_OUT={Z_OUT}")
fig_z.add_hline(y=0, line_dash="dash")
fig_z.add_hline(y= Z_IN,  line_dash="dot")
fig_z.add_hline(y=-Z_IN,  line_dash="dot")
fig_z.add_hline(y= Z_OUT, line_dash="dashdot")
fig_z.add_hline(y=-Z_OUT, line_dash="dashdot")
fig_z.show()


In [20]:
# %%
import numpy as np
import pandas as pd
import plotly.express as px

# thresholds (match your plot)
#Z_IN  = 1.8
#Z_OUT = 0.1

# optional: per-leg cost in spread units (for quick what-if)
COST_PER_LEG = 0.0   # set e.g. 0.02 in spread units to include costs

# state machine
pos = 0   # +1 = long spread (long SYM1, short β·SYM2); -1 = short spread; 0 = flat
trades = []
entry_i = None
entry_spread = None
entry_z = None

equity = np.zeros(len(df))
last_spread = df["spread"].iloc[0]

for i, r in enumerate(df.itertuples(index=False)):
    s = r.spread
    z = r.zscore

    # mark to market (P&L comes from change in spread when sized β-neutral)
    ds = s - last_spread
    if np.isfinite(ds):
        equity[i] = equity[i-1] + pos * ds
    else:
        equity[i] = equity[i-1]
    last_spread = s

    # signals
    if np.isfinite(z):
        if pos == 0:
            if z >= Z_IN:
                pos = -1
                entry_i = i; entry_spread = s; entry_z = z
                equity[i] -= 2 * COST_PER_LEG
                trades.append({"time": r.time, "i": i, "action": "ENTER_SHORT", "z": z, "spread": s})
            elif z <= -Z_IN:
                pos = +1
                entry_i = i; entry_spread = s; entry_z = z
                equity[i] -= 2 * COST_PER_LEG
                trades.append({"time": r.time, "i": i, "action": "ENTER_LONG", "z": z, "spread": s})
        else:
            if abs(z) < Z_OUT:
                # exit to flat
                pos = 0
                equity[i] -= 2 * COST_PER_LEG
                trades.append({"time": r.time, "i": i, "action": "EXIT", "z": z, "spread": s,
                               "pnl_spread": s - entry_spread if entry_spread is not None else np.nan,
                               "hold_bars": i - (entry_i or i)})
                entry_i = entry_spread = entry_z = None

# compile results
trades_df = pd.DataFrame(trades)
display(trades_df.tail())

# quick KPIs in spread units
wins = trades_df.query("action=='EXIT' and pnl_spread.notna() and pnl_spread!=0")
hit = (np.sign(wins["pnl_spread"]).gt(0)).mean() if len(wins) else np.nan
avg_pnl = wins["pnl_spread"].mean() if len(wins) else np.nan
med_hold = wins["hold_bars"].median() if len(wins) else np.nan
print(f"Trades: {len(wins)} | Hit rate: {hit:.1%} | Avg PnL (spread units): {avg_pnl:.3f} | Median hold (bars): {med_hold}")

# equity plot
df["equity_spread_units"] = equity
px.line(df, x="time", y="equity_spread_units",
        title="Cumulative P&L (spread units)").show()


,time,i,action,z,spread,pnl_spread,hold_bars
61,2025-09-26 10:00:00,4346,EXIT,0.492890,80.154481,-67.676984,13.0
62,2025-10-03 18:00:00,4469,ENTER_LONG,-2.304554,-83.802273,NaN,NaN
63,2025-10-06 14:00:00,4488,EXIT,-0.057200,14.909073,98.711346,19.0
64,2025-10-08 23:00:00,4543,ENTER_SHORT,2.327550,93.348060,NaN,NaN
65,2025-10-14 07:00:00,4619,EXIT,0.459362,76.573973,-16.774087,76.0


Trades: 33 | Hit rate: 45.5% | Avg PnL (spread units): -9.975 | Median hold (bars): 37.0


### quick grid search to found good z_in and z_out params although the comlpete grid search will do in mt5 built in backtester

In [21]:
# %% quick grid search for Z_in / Z_out
import numpy as np
import pandas as pd

def backtest_spread(df, Z_IN, Z_OUT, cost_per_leg=0.0):
    pos = 0
    equity = 0.0
    last = None
    pnls = []
    trades = 0
    for r in df.itertuples(index=False):
        s = getattr(r, "spread", np.nan)
        z = getattr(r, "zscore", np.nan)
        if np.isfinite(s):
            if last is not None:
                equity += pos * (s - last)
                pnls.append(pos * (s - last))
            last = s
        if not np.isfinite(z):
            continue
        if pos == 0:
            if z >= Z_IN:
                pos = -1; equity -= 2*cost_per_leg; trades += 1
            elif z <= -Z_IN:
                pos = +1; equity -= 2*cost_per_leg; trades += 1
        else:
            if abs(z) < Z_OUT:
                pos = 0; equity -= 2*cost_per_leg
    pnls = np.array(pnls, dtype=float)
    if len(pnls) < 3: 
        return {"pnl": equity, "sharpe": 0.0, "trades": trades}
    mean = pnls.mean()
    std  = pnls.std(ddof=1)
    sharpe = (mean/std)*np.sqrt(24*5) if std>0 else 0.0   # scale: H1≈24*5 bars/week
    return {"pnl": equity, "sharpe": sharpe, "trades": trades}

ZINs  = np.arange(0.8, 3.0, 0.1)   # 1.5 … 2.5
ZOUTs = np.arange(0.1, 1.5, 0.1) # 0.15 … 0.60
COST  = 0.0  # set to your per-leg cost in spread units

rows = []
for zin in ZINs:
    for zout in ZOUTs:
        if zout >= zin: 
            continue
        m = backtest_spread(df, zin, zout, cost_per_leg=COST)
        rows.append({"Z_in": zin, "Z_out": zout, **m})
grid = pd.DataFrame(rows).sort_values(["sharpe","pnl"], ascending=[False,False])
display(grid.head(20))


,Z_in,Z_out,pnl,sharpe,trades
156,2.1,0.3,3170.945456,0.702465,30
169,2.2,0.2,3079.021785,0.698308,27
170,2.2,0.3,3015.858761,0.691160,27
60,1.4,0.4,3708.225220,0.687320,56
183,2.3,0.2,2970.461546,0.681343,26
72,1.5,0.3,3600.968010,0.679517,50
59,1.4,0.3,3718.156566,0.676791,53
184,2.3,0.3,2907.298523,0.674018,26
114,1.8,0.3,3308.962279,0.671019,38
128,1.9,0.3,3244.238537,0.670823,36
